This script process DNA and RNA  integration results and merge them together side by side. It is written for Cervical project.

In [17]:
import matplotlib
import seaborn
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import scipy.stats as ss
from matplotlib import style
matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (15, 9)
import pandas as pd
# figure out number of indels

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

This function process groupby object, combine all rows with the same patient/virus indexed entries into one row

In [62]:
def process_grp(grp):
#     _df_or_sr = grp.apply(lambda col: ';'.join(col.unique().tolist()))
#     lambda x: True if x % 2 == 0 else False
    _df_or_sr = grp.apply(lambda col: ';'.join(col.unique().tolist()) if len(col.unique())==1 else ';'.join(col.tolist()))
    _df = _df_or_sr.to_frame().T
    assert _df.shape[0] == 1
    return _df

take in concatenated integration results and add in patient ids

In [63]:
# rna integration results for 19 new libraries
# f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/101_patients/19_integration_results.tmp'
# f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/107_patients/6_new_RNA_integration_results.txt'
f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/82_patients/transcriptome/transcriptome_integration_summary.csv'
itg = pd.read_csv(f1, sep='\t')
itg.head(2)

,library,virus,Integration,human-breakpoint,human-gene(s)
0,A37700,Human papillomavirus type 16,YES,3_189526056,TP63
1,A37700,Human papillomavirus type 16,YES,3_189526059,TP63


In [65]:
# f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/101_patients/19_new_libs.txt'
# f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/107_patients/6_new_RNAs.txt'
f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/82_patients/transcriptome/82_RNAs.txt'
pat = pd.read_csv(f2, sep='\t', header=None, names=['patient', 'source_id', 'library'])

pat.head()

,patient,source_id,library
0,HTMCP-03-06-02001,HTMCP_124,A37700
1,HTMCP-03-06-02002,HTMCP_125,A37701
2,HTMCP-03-06-02003,HTMCP_126,A37702
3,HTMCP-03-06-02006,HTMCP_129,A37704
4,HTMCP-03-06-02007,HTMCP_130,A37705


In [66]:
itgr = pd.merge(itg, pat, left_on='library', right_on='library')
itgr.head()

,library,virus,Integration,human-breakpoint,human-gene(s),patient,source_id
0,A37700,Human papillomavirus type 16,YES,3_189526056,TP63,HTMCP-03-06-02001,HTMCP_124
1,A37700,Human papillomavirus type 16,YES,3_189526059,TP63,HTMCP-03-06-02001,HTMCP_124
2,A37700,Human papillomavirus type 16,YES,3_189526315,TP63,HTMCP-03-06-02001,HTMCP_124
3,A37700,Human papillomavirus type 16,YES,3_189526317,TP63,HTMCP-03-06-02001,HTMCP_124
4,A37700,Human papillomavirus type 16,YES,3_189542597,TP63,HTMCP-03-06-02001,HTMCP_124


In [67]:
print(itgr.shape)
itgr = itgr[itgr['virus'] != 'No-Hits']
print(itgr.shape)
itgr.head()

(520, 7)
(513, 7)


,library,virus,Integration,human-breakpoint,human-gene(s),patient,source_id
0,A37700,Human papillomavirus type 16,YES,3_189526056,TP63,HTMCP-03-06-02001,HTMCP_124
1,A37700,Human papillomavirus type 16,YES,3_189526059,TP63,HTMCP-03-06-02001,HTMCP_124
2,A37700,Human papillomavirus type 16,YES,3_189526315,TP63,HTMCP-03-06-02001,HTMCP_124
3,A37700,Human papillomavirus type 16,YES,3_189526317,TP63,HTMCP-03-06-02001,HTMCP_124
4,A37700,Human papillomavirus type 16,YES,3_189542597,TP63,HTMCP-03-06-02001,HTMCP_124


In [68]:
RNA_GRP = itgr.groupby(['patient', 'virus']).apply(process_grp).reset_index(level=2, drop=True)
del RNA_GRP['patient']
del RNA_GRP['virus']
# RNA_GRP
RNA_GRP.reset_index(inplace=True)

print(itgr.head(3))

# RNA_GRP.head()

header_dict = {i:'_'.join(['RNA', i]) for i in RNA_GRP.columns.tolist()}

RNA_GRP.rename(columns=header_dict, inplace=True)

RNA_GRP.head()

  library                         virus Integration human-breakpoint  \
0  A37700  Human papillomavirus type 16         YES      3_189526056   
1  A37700  Human papillomavirus type 16         YES      3_189526059   
2  A37700  Human papillomavirus type 16         YES      3_189526315   

  human-gene(s)            patient  source_id  
0          TP63  HTMCP-03-06-02001  HTMCP_124  
1          TP63  HTMCP-03-06-02001  HTMCP_124  
2          TP63  HTMCP-03-06-02001  HTMCP_124  


,RNA_patient,RNA_virus,RNA_library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s),RNA_source_id
0,HTMCP-03-06-02001,Human papillomavirus type 16,A37700,YES,3_189526056;3_189526059;3_189526315;3_18952631...,TP63,HTMCP_124
1,HTMCP-03-06-02002,Human papillomavirus type 52,A37701,YES,22_43161525;22_43161526;22_43163674;22_4316513...,intergenic,HTMCP_125
2,HTMCP-03-06-02003,Human papillomavirus type 16,A37702,YES,11_60882522;11_60935033,CD5;intergenic,HTMCP_126
3,HTMCP-03-06-02006,Human papillomavirus type 18,A37704,NO,-,-,HTMCP_129
4,HTMCP-03-06-02006,Human papillomavirus type 45,A37704,YES,12_46287201;12_46328672;12_46329948;12_4633703...,ARID2;SCAF11;SCAF11;SCAF11;SCAF11;SCAF11,HTMCP_129


In [69]:
# dna integration results for 19 new libraries
# f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/101_patients/19_new_genomes_integration_results.txt'
# f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/107_patients/6_new_genomes_integration_results.txt'
f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/82_patients/genome/genome_integration_summary.csv'
itg = pd.read_csv(f1, sep='\t')
itg.head(2)

# f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/101_patients/19_new_genomes.txt'
# f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/107_patients/6_new_genomes.txt'
f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/82_patients/genome/82_genomes.txt'
pat = pd.read_csv(f2, sep='\t', header=None, names=['patient', 'source_id', 'library'])

pat.head(2)

itgr = pd.merge(itg, pat, left_on='library', right_on='source_id')
itgr.head(2)



print(itgr.shape)
itgr = itgr[itgr['virus'] != 'No-Hits']
print(itgr.shape)
# itgr

DNA_GRP = itgr.groupby(['patient', 'virus']).apply(process_grp).reset_index(level=2, drop=True)
del DNA_GRP['patient']
del DNA_GRP['virus']
# DNA_GRP
DNA_GRP.reset_index(inplace=True)

print(itgr.head(3))

DNA_GRP.head()

header_dict = {i:'_'.join(['DNA', i]) for i in DNA_GRP.columns.tolist()}

DNA_GRP.rename(columns=header_dict, inplace=True)

DNA_GRP.head()

(271, 8)
(226, 8)
   library_x                         virus Integration human-breakpoint  \
2  HTMCP_126  Human papillomavirus type 16          NO                -   
6  HTMCP_133  Human papillomavirus type 73         YES      17_36310359   
7  HTMCP_133  Human papillomavirus type 34          NO                -   

  human-gene(s)            patient  source_id library_y  
2             -  HTMCP-03-06-02003  HTMCP_126    A37236  
6    intergenic  HTMCP-03-06-02012  HTMCP_133    A37243  
7             -  HTMCP-03-06-02012  HTMCP_133    A37243  


,DNA_patient,DNA_virus,DNA_library_x,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),DNA_source_id,DNA_library_y
0,HTMCP-03-06-02003,Human papillomavirus type 16,HTMCP_126,NO,-,-,HTMCP_126,A37236
1,HTMCP-03-06-02012,Human papillomavirus type 34,HTMCP_133,NO,-,-,HTMCP_133,A37243
2,HTMCP-03-06-02012,Human papillomavirus type 73,HTMCP_133,YES,17_36310359,intergenic,HTMCP_133,A37243
3,HTMCP-03-06-02036,Human papillomavirus type 16,HTMCP_142,NO,-,-,HTMCP_142,A37252
4,HTMCP-03-06-02037,Human papillomavirus type 52,HTMCP_143,NO,-,-,HTMCP_143,A37253


In [52]:

# DNA_GRP.set_index(['DNA_patient', 'DNA_virus'],inplace=True)

In [70]:
DNA_GRP.head()

,DNA_patient,DNA_virus,DNA_library_x,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),DNA_source_id,DNA_library_y
0,HTMCP-03-06-02003,Human papillomavirus type 16,HTMCP_126,NO,-,-,HTMCP_126,A37236
1,HTMCP-03-06-02012,Human papillomavirus type 34,HTMCP_133,NO,-,-,HTMCP_133,A37243
2,HTMCP-03-06-02012,Human papillomavirus type 73,HTMCP_133,YES,17_36310359,intergenic,HTMCP_133,A37243
3,HTMCP-03-06-02036,Human papillomavirus type 16,HTMCP_142,NO,-,-,HTMCP_142,A37252
4,HTMCP-03-06-02037,Human papillomavirus type 52,HTMCP_143,NO,-,-,HTMCP_143,A37253


In [54]:
# RNA_GRP.set_index(['RNA_patient', 'RNA_virus'],inplace=True)

In [71]:
RNA_GRP.head()

,RNA_patient,RNA_virus,RNA_library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s),RNA_source_id
0,HTMCP-03-06-02001,Human papillomavirus type 16,A37700,YES,3_189526056;3_189526059;3_189526315;3_18952631...,TP63,HTMCP_124
1,HTMCP-03-06-02002,Human papillomavirus type 52,A37701,YES,22_43161525;22_43161526;22_43163674;22_4316513...,intergenic,HTMCP_125
2,HTMCP-03-06-02003,Human papillomavirus type 16,A37702,YES,11_60882522;11_60935033,CD5;intergenic,HTMCP_126
3,HTMCP-03-06-02006,Human papillomavirus type 18,A37704,NO,-,-,HTMCP_129
4,HTMCP-03-06-02006,Human papillomavirus type 45,A37704,YES,12_46287201;12_46328672;12_46329948;12_4633703...,ARID2;SCAF11;SCAF11;SCAF11;SCAF11;SCAF11,HTMCP_129


In [72]:
# rename patient and virus to common names for outer merge. 
# not doing this step results in 2 columns for each data set, unique entries for each set will be all na
# itg_merge = pd.merge(DNA_GRP, RNA_GRP, left_on=['DNA_patient', 'DNA_virus'], how='outer',right_on=['RNA_patient', 'RNA_virus'])
DNA_GRP = DNA_GRP.rename(columns={'DNA_patient': 'patient', 'DNA_virus':'virus', 'DNA_source_id':'source_id'})
RNA_GRP = RNA_GRP.rename(columns={'RNA_patient': 'patient', 'RNA_virus':'virus', 'RNA_source_id':'source_id'})
itg_merge = pd.merge(DNA_GRP, RNA_GRP, on=['patient', 'source_id', 'virus'], how='outer')

In [73]:
itg_merge.fillna('na', inplace=True)

In [74]:
itg_merge.drop('DNA_library_x', inplace=True, axis=1)

In [75]:
itg_merge

,patient,virus,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),source_id,DNA_library_y,RNA_library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s)
0,HTMCP-03-06-02003,Human papillomavirus type 16,NO,-,-,HTMCP_126,A37236,A37702,YES,11_60882522;11_60935033,CD5;intergenic
1,HTMCP-03-06-02012,Human papillomavirus type 34,NO,-,-,HTMCP_133,A37243,A37707,NO,-,-
2,HTMCP-03-06-02012,Human papillomavirus type 73,YES,17_36310359,intergenic,HTMCP_133,A37243,A37707,YES,17_36285541;17_36347084;17_36401125;6_92985326,"TBC1D3;TBC1D3,LOC440434;LOC440434;intergenic"
3,HTMCP-03-06-02036,Human papillomavirus type 16,NO,-,-,HTMCP_142,A37252,A37714,NO,-,-
4,HTMCP-03-06-02037,Human papillomavirus type 52,NO,-,-,HTMCP_143,A37253,A37715,YES,6_157714033;6_157731388;6_157731411;6_15773531...,TMEM242
5,HTMCP-03-06-02042,Human papillomavirus type 16,YES,1_151029494;1_151029496,CDC42SE1,HTMCP_145,A37255,A37717,YES,1_150997084;1_150997085;1_150997988;1_15099970...,PRUNE;PRUNE;PRUNE;PRUNE;CDC42SE1;CDC42SE1;CDC4...
6,HTMCP-03-06-02046,Human papillomavirus type 16,YES,19_1181971;19_1183502;19_1192727,intergenic,HTMCP_147,A37257,A37718,YES,19_1181971;19_1182527;19_1183716;19_1184195;19...,intergenic;intergenic;intergenic;intergenic;in...
7,HTMCP-03-06-02054,Human papillomavirus type 52,NO,-,-,HTMCP_149,A37259,A37720,YES,14_87398614;1_93913863;1_93926370;8_42394076;8...,intergenic;FNBP1L;FNBP1L;SLC20A2;SLC20A2;SLC20...
8,HTMCP-03-06-02058,Human papillomavirus type 18,YES,18_37408866;8_128319365;8_128323246,"intergenic;DQ515899,DQ515898,LOC727677;DQ51589...",HTMCP_151,A37261,A37722,YES,8_128289076;8_128302305;8_128302307;8_12830230...,"intergenic;DQ515899,DQ515898,LOC727677;DQ51589..."
9,HTMCP-03-06-02063,Human papillomavirus type 16,YES,2_141933994;2_142008771;22_42896541,LRP1B;LRP1B;intergenic,HTMCP_156,A37266,A37725,YES,2_142008770;2_142008771;2_142009078;2_14201026...,LRP1B;LRP1B;LRP1B;LRP1B;LRP1B;SERHL2;SERHL2


In [77]:
dft = itg_merge[['patient', 'virus', 'DNA_Integration', 'RNA_Integration']]

In [78]:
dft

,patient,virus,DNA_Integration,RNA_Integration
0,HTMCP-03-06-02003,Human papillomavirus type 16,NO,YES
1,HTMCP-03-06-02012,Human papillomavirus type 34,NO,NO
2,HTMCP-03-06-02012,Human papillomavirus type 73,YES,YES
3,HTMCP-03-06-02036,Human papillomavirus type 16,NO,NO
4,HTMCP-03-06-02037,Human papillomavirus type 52,NO,YES
5,HTMCP-03-06-02042,Human papillomavirus type 16,YES,YES
6,HTMCP-03-06-02046,Human papillomavirus type 16,YES,YES
7,HTMCP-03-06-02054,Human papillomavirus type 52,NO,YES
8,HTMCP-03-06-02058,Human papillomavirus type 18,YES,YES
9,HTMCP-03-06-02063,Human papillomavirus type 16,YES,YES


In [80]:
of3 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/82_patients/82_patient_merged_integration_results_concensus.tmp'
dft.set_index(['virus','patient']).to_csv(of3)

In [76]:
# f4 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/101_patients/19_patient_merged_integration_results.txt'
f4 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/82_patients/82_patient_merged_integration_results.txt'
itg_merge.to_csv(f4, sep='\t', index=False)

In [93]:
# add in HPVdetector results
f3 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/101_patients/hpvdetector_results_19_patients.txt'
hpvd = pd.read_csv(f3, sep='\t')
hpvd.head(2)

,virus,patient,integration
0,HPV_18,HTMCP-03-06-02182,MAYBE
1,HPV_31,HTMCP-03-06-02197,MAYBE


In [105]:
hpvd.rename(columns={'integration':'HPVdetector_integration'}, inplace=True)

In [101]:
# itg_merge.virus.apply(lambda x: re.sub('Human papillomavirus type', 'HPV', x)).apply(lambda y: re.sub('Human papillomavirus isolate ', '', y))

In [102]:
f5 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/101_patients/19_patient_merged_integration_results.xls'
itg_sum = pd.read_excel(f5, worksheets='worksheet1')
itg_sum.head()

,patient,virus,DNA_Integration,RNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),source_id,DNA_library_y,RNA_library,RNA_human-breakpoint,RNA_human-gene(s)
0,HTMCP-03-06-02197,HPV_31,YES,YES,16_19763518;16_19764226,IQCK,HTMCP_468,A62912,A62930,16_19755127;16_19758194;16_19763518;16_1976422...,IQCK
1,HTMCP-03-06-02222,HPV_51,NO,NO,-,-,HTMCP_491,A68231,A68250,-,-
2,HTMCP-03-06-02222,HPV_82,YES,YES,2_3514512,ADI1,HTMCP_491,A68231,A68250,2_3502662;2_3504637;2_3504766;2_3513909;2_3514...,ADI1
3,HTMCP-03-06-02256,HPV_45,YES,YES,18_61468505;18_61545616,SERPINB7;intergenic,HTMCP_493,A68232,A68251,18_61468505;18_61471469,SERPINB7
4,HTMCP-03-06-02259,HPV_45,YES,YES,1_121484601,intergenic,HTMCP_495,A68233,A68252,4_68772816;4_68812291;4_68823543;4_68832841,"LOC550112;LOC550112,TMPRSS11A;LOC550112,TMPRSS..."


In [106]:
mg = pd.merge(itg_sum, hpvd, on=['patient', 'virus'], how='outer')

In [109]:
mg.fillna('na', inplace=True)

In [111]:
f6 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/101_patients/combined_dna_rna_hpvdetector_results.csv'
mg.to_csv(f6, index=False)

In [127]:
'{0} == "{1}"'.format('DNA_Integration','YES')

'DNA_Integration == "YES"'

In [136]:
# mg.query('{0} == "{1}"'.format('DNA_Integration','YES'))
pat_with_hpv = mg.query(('DNA_Integration == "YES" | DNA_Integration == "NO" |RNA_Integration == "YES" | RNA_Integration == "NO"')).patient

In [138]:
pat_with_hpv.nunique()

19

In [4]:
RNA_integration = pd.read_csv('/projects/trans_scratch/validations/workspace/szong/Cervical/integration/82_patients/genome/genome_integration_summary.csv.added', sep='\t')
print(RNA_integration.shape)
RNA_integration = RNA_integration[RNA_integration['virus'] != 'No-Hits']
print(RNA_integration.shape)
# RNA_integration

RNA_GRP = RNA_integration.groupby(['patient', 'virus']).apply(process_grp).reset_index(level=2, drop=True)
del RNA_GRP['patient']
del RNA_GRP['virus']
# RNA_GRP
RNA_GRP.reset_index(inplace=True)

(271, 7)
(226, 7)


In [5]:
DNA_integration = pd.read_csv('/projects/trans_scratch/validations/workspace/szong/Cervical/integration/82_patients/transcriptome/transcriptome_integration_summary.csv.added', sep='\t')
print(DNA_integration.shape)
DNA_integration = DNA_integration[DNA_integration['virus'] != 'No-Hits']
print(DNA_integration.shape)

# DNA_integration.groupby(['patient', 'virus']).apply(process_grp).head().reset_index(level=2, drop=True)
DNA_GRP = DNA_integration.groupby(['patient', 'virus']).apply(process_grp).reset_index(level=2, drop=True)
del DNA_GRP['patient']
del DNA_GRP['virus']
# DNA_GRP
DNA_GRP.reset_index(inplace=True)

(520, 6)
(513, 6)


In [9]:
merged = pd.merge(DNA_GRP, RNA_GRP, on=['patient', 'virus'], how='outer', suffixes=('_RNA', '_DNA'))
merged.fillna('na', inplace=True)
merged.to_csv('test_final_merged.csv', sep='\t')
merged.head()

np.size(merged.patient.unique())
strains = merged.virus.unique()
strains


meta = pd.read_csv('/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/82_patients/HIV_Cervical_82_pateints_vcfs_bams.csv', sep='\t')
all_patients = meta.patient.unique()
np.size(all_patients)
merged.set_index(['patient', 'virus'], inplace=True)

df_patients = list(set([i[0] for i in merged.index.unique().tolist()]))

add entries for all patients and strains for those absent from the summary file


In [10]:
for patient in all_patients:
    df_strains = []
    if patient in df_patients:
        df_strains = merged.loc[(patient)].index.unique().tolist()
        for strain in strains:
            if strain not in df_strains:
                merged.loc[(patient, strain),:] = pd.Series({i:'bbbb' for i in merged.columns}) 
    else:
        for strain in strains:
            merged.loc[(patient, strain),:] = pd.Series({i:'bbbb' for i in merged.columns})

In [261]:
merged.to_csv('final_final_test.csv', sep='\t')

In [ ]:
# below are experiment code

In [227]:
merged.loc[('HTMCP-03-06-02225')].index.get_level_values('virus').unique().tolist()
# multiindex is a tuple
merged.loc[('HTMCP-03-06-02225','Human papillomavirus type 69')]
# list comprehension to make a dictionary and then to series
pd.Series({i:'bbbb' for i in merged.columns})
# add a entry for multiindexed data frame
merged.loc[('HTMCP-03-06-02225', 'Human papillomavirus type 999'),:] = pd.Series({i:'not_detected' for i in merged.columns})

# add all negative patients
df_patients = list(set([i[0] for i in merged.index.unique().tolist()]))

['Human papillomavirus type 51',
 'Human papillomavirus type 69',
 'Human papillomavirus type 82']

In [25]:
# merged['virus', 'patient', 'Integration_RNA']
merged['Integration_RNA'].

patient            virus                       
HTMCP-03-06-02001  Human papillomavirus type 16     YES
HTMCP-03-06-02002  Human papillomavirus type 52     YES
HTMCP-03-06-02003  Human papillomavirus type 16     YES
HTMCP-03-06-02006  Human papillomavirus type 18      NO
                   Human papillomavirus type 45     YES
HTMCP-03-06-02008  Human papillomavirus type 16      NO
                   Human papillomavirus type 9      YES
HTMCP-03-06-02012  Human papillomavirus type 16     YES
                   Human papillomavirus type 18      NO
                   Human papillomavirus type 34      NO
                   Human papillomavirus type 58      NO
                   Human papillomavirus type 73     YES
HTMCP-03-06-02013  Human papillomavirus type 16     YES
HTMCP-03-06-02020  Human papillomavirus type 18      NO
                   Human papillomavirus type 45     YES
HTMCP-03-06-02036  Human papillomavirus type 16      NO
HTMCP-03-06-02037  Human papillomavirus type 52     YES


### figure out the integrated viral genes

In [21]:
idxs = np.where(A >= 0)
sorted_A = sorted(A[idxs])
sorted_A

[3, 5, 8, 9, 12]

In [43]:
def solution(A):
    # write your code in Python 3.6
    # remove negative integers
    A.sort()
    A =[i for i in A if i >=0]
    A = list(set(A))
    N = len(sorted_A)
    for i in range(N):
        for j in range(i+1, N, 1):
            for k in range(j+1, N, 1):    
                P = sorted_A[i]
                Q = sorted_A[j]
                R = sorted_A[k]
                if (P + Q > R):
                    print(P, Q, R)
                    return 1
    return 0

In [44]:
solution(A)

2 3 4


1

In [56]:
A = [5, 8, -1, 3, 3,  9, 0, 2, 4, 12]
A.sort()
A
B =[i for i in A if i >=0]
B

[-1, 0, 2, 3, 3, 4, 5, 8, 9, 12]

[0, 2, 3, 3, 4, 5, 8, 9, 12]

In [60]:
solution(A)

1